## State Student Spending Case Study
### Examining any relationship between student sucess or lack thereof with spending


In [1]:
#import necessry modules

import pandas as pd

In [2]:
#import data
Grad = pd.read_csv('Data/Completion1318edit.csv')
Drop = pd.read_csv('Data/Dropout1318edit.csv')
StateSpend = pd.read_excel('Data/StateSpendingStudents.xlsx')
Pop = pd.read_csv('Data/Pop10-19.csv')

In [3]:
#examine data
print(Grad.columns)

Index(['State', '2012-2013', '2013-2014', '2014-2015', '2015-2016',
       '2016-2017', '2017-2018'],
      dtype='object')


In [4]:
#Rename columns so they match years in other files.
oldname = Grad.columns
newname = ['State', 2013, 2014, 2015, 2016, 2017, 2018 ]
Grad = Grad.rename(columns=dict(zip(oldname, newname)))

#Flip so States are columns
columnname = Grad['State']
Grad = pd.pivot_table(Grad, columns = columnname)

#Examine
Grad.head()

State,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,United States,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2013,80.0,71.8,75.1,84.9,80.4,76.9,85.5,80.4,62.3,75.6,...,86.3,88.0,81.4,83.0,86.6,84.5,76.4,81.4,88.0,77.0
2014,86.3,71.1,75.7,86.9,81.0,77.3,87.0,87.0,61.4,76.1,...,87.2,88.3,82.3,83.9,87.8,85.3,78.2,84.5,88.6,78.6
2015,89.3,75.6,77.4,84.9,82.0,77.3,87.2,85.6,68.5,77.9,...,87.9,89.0,83.2,84.8,87.7,85.7,78.2,86.5,88.4,79.3
2016,87.1,76.1,79.5,87.0,83.0,78.9,87.4,85.5,69.2,80.7,...,88.5,89.1,84.1,85.2,87.7,86.7,79.7,89.8,88.2,80.0
2017,89.3,78.2,78.0,88.0,82.7,79.1,87.9,86.9,73.2,82.3,...,89.8,89.7,84.6,86.0,89.1,86.9,79.4,89.4,88.6,86.2


In [5]:
#examine data
print(Drop.head())
print(Drop.columns)

           State 2013 2014 2015  2016 2017 2018
0  United States  6.8  6.3  6.0   5.8  5.4  5.3
1        Alabama  8.9  7.7  7.4   6.9  6.6  5.4
2         Alaska  6.2  2.8  8.8   5.6  5.3  4.4
3        Arizona  9.4  8.5  9.2   7.6  8.0  7.7
4       Arkansas  6.8  6.8  6.7   7.2  5.7  5.6
Index(['State', '2013', '2014', '2015', '2016', '2017', '2018'], dtype='object')


In [6]:
#Flip so States are columns
columnname = Drop['State']
Drop = pd.pivot_table(Drop, columns = columnname)

#Examine
Drop.head()

#QUESTION FOR MENTOR: Why am I only gettin 2016?

State,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,United States,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2016,6.9,5.6,7.6,7.2,5.3,6.4,3.1,7.1,3.5,6.6,...,4.8,6.7,5.8,4.7,4.5,3.9,7.5,6.6,4.7,5.2


In [7]:
#Examine State Spending
print(StateSpend.head())

        state      1997      1998      1999      2000        2001      2002  \
0     Alabama   3271969   3504764   3784422   4006894   4140052.5   4273211   
1      Alaska   1042311   1019614   1097282   1133145   1188460.5   1243776   
2     Arizona   3388165   3579363   3849420   4185930   4427480.5   4669031   
3    Arkansas   1960613   2127517   2224967   2324123   2451202.0   2578281   
4  California  28708364  31776920  33739312  36658880  41025508.0  45392132   

         2003      2004      2005  ...      2007      2008      2009  \
0   4429788.5   4586366   4925201  ...   6104470   6671430   6629154   
1   1287792.0   1331808   1416771  ...   1609832   1897947   2077152   
2   5087547.5   5506064   5872686  ...   6826134   7367909   7536710   
3   2742677.5   2907074   3300288  ...   3719097   3846131   4189066   
4  47133896.0  48875660  50693044  ...  57258252  60096496  59255344   

       2010      2011      2012      2013      2014      2015      2016  
0   6573857   6592

In [8]:
#Realign with States as columns
columnname = StateSpend['state']
StateSpend = pd.pivot_table(StateSpend, columns = columnname)

#Examine
print(StateSpend.head())

state    Alabama     Alaska    Arizona   Arkansas  California   Colorado  \
1997   3271969.0  1042311.0  3388165.0  1960613.0  28708364.0  3332994.0   
1998   3504764.0  1019614.0  3579363.0  2127517.0  31776920.0  3684502.0   
1999   3784422.0  1097282.0  3849420.0  2224967.0  33739312.0  3916260.0   
2000   4006894.0  1133145.0  4185930.0  2324123.0  36658880.0  4149462.0   
2001   4140052.5  1188460.5  4427480.5  2451202.0  41025508.0  4487114.0   

state  Connecticut  Delaware  District of Columbia     Florida  ...  \
1997     4014870.0  776825.0              544051.0  11498394.0  ...   
1998     3931428.0  815900.0              623415.0  12141943.0  ...   
1999     4455325.0  848633.0              685988.0  12987313.0  ...   
2000     4713226.0  915317.0              815299.0  13255884.0  ...   
2001     5027140.0  974989.5              910714.5  14015765.0  ...   

state  South Dakota  Tennessee       Texas       Utah   Vermont   Virginia  \
1997       610507.0  3880074.0  181197

In [9]:
#Limit to years of interest
StateSpendTrain = StateSpend.iloc[16:20]

#State Spend Documentation ends after 2016
StateSpendTest = StateSpend.iloc[19:]

print(StateSpendTrain)
print(StateSpendTest)

state    Alabama     Alaska    Arizona   Arkansas  California   Colorado  \
2013   6461916.0  2202551.0  6675200.0  4222408.0  57043384.0  7011149.0   
2014   6600333.0  2202685.0  7065613.0  4299286.0  59043948.0  7405899.0   
2015   6645129.0  2332630.0  7026131.0  4355246.0  64553292.0  7689567.0   
2016   6589890.0  2074993.0  7137123.0  4411761.0  68892072.0  8060420.0   

state  Connecticut   Delaware  District of Columbia     Florida  ...  \
2013     8009846.0  1674823.0             2006747.0  22016268.0  ...   
2014     8310423.0  1721283.0             2058261.0  23146180.0  ...   
2015     8510423.0  1764887.0             2152055.0  23815164.0  ...   
2016     8687640.0  1832385.0             2290288.0  24222830.0  ...   

state  South Dakota  Tennessee       Texas       Utah    Vermont    Virginia  \
2013      1057858.0  7896419.0  38458788.0  3677422.0  1442269.0  13729023.0   
2014      1114420.0  8228575.0  40392404.0  3685048.0  1471609.0  14151467.0   
2015      1144394.

In [10]:
#Examine the Population
print(Pop.head(10))
print(Pop['Geographic Area'].head(10))
#print(Pop.rows)

  Geographic Area       Census Estimates Base         2010         2011  \
0   United States  308,745,538    308,758,105  309,321,666  311,556,874   
1       Northeast   55,317,240     55,318,443   55,380,134   55,604,223   
2         Midwest   66,927,001     66,929,725   66,974,416   67,157,800   
3           South  114,555,744    114,563,030  114,866,680  116,006,522   
4            West   71,945,553     71,946,907   72,100,436   72,788,329   
5        .Alabama    4,779,736      4,780,125    4,785,437    4,799,069   
6         .Alaska      710,231        710,249      713,910      722,128   
7        .Arizona    6,392,017      6,392,288    6,407,172    6,472,643   
8       .Arkansas    2,915,918      2,916,031    2,921,964    2,940,667   
9     .California   37,253,956     37,254,519   37,319,502   37,638,369   

          2012         2013         2014         2015         2016  \
0  313,830,990  315,993,715  318,301,008  320,635,163  322,941,311   
1   55,775,216   55,901,806   56,0

In [11]:
#Fix geographic area names
Pop['Geographic Area'] = Pop['Geographic Area'].str.replace('.','')
print(Pop['Geographic Area'].head(10))

0    United States
1        Northeast
2          Midwest
3            South
4             West
5          Alabama
6           Alaska
7          Arizona
8         Arkansas
9       California
Name: Geographic Area, dtype: object


In [12]:
#Drop the US regions
Pop = Pop.drop([0,1,2,3,4,])
Pop = Pop.reset_index(drop = True)
Pop.head(10)

,Geographic Area,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,"4,779,736","4,780,125","4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185"
1,Alaska,"710,231","710,249","713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545"
2,Arizona,"6,392,017","6,392,288","6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717"
3,Arkansas,"2,915,918","2,916,031","2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804"
4,California,"37,253,956","37,254,519","37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223"
5,Colorado,"5,029,196","5,029,319","5,047,349","5,121,108","5,192,647","5,269,035","5,350,101","5,450,623","5,539,215","5,611,885","5,691,287","5,758,736"
6,Connecticut,"3,574,097","3,574,147","3,579,114","3,588,283","3,594,547","3,594,841","3,594,524","3,587,122","3,578,141","3,573,297","3,571,520","3,565,287"
7,Delaware,"897,934","897,937","899,593","907,381","915,179","923,576","932,487","941,252","948,921","956,823","965,479","973,764"
8,District of Columbia,"601,723","601,767","605,226","619,800","634,924","650,581","662,328","675,400","685,815","694,906","701,547","705,749"
9,Florida,"18,801,310","18,804,564","18,845,537","19,053,237","19,297,822","19,545,621","19,845,911","20,209,042","20,613,477","20,963,613","21,244,317","21,477,737"


In [16]:
#Clean up last bit of nonesense at bottom of file
Pop = Pop.drop([51,52,53,54,55,56,57])
print(Pop['Geographic Area'])


0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Ohio
36                Oklahoma
3

In [19]:
#Flip Columns
columnname = Pop['Geographic Area']
print(columnname)
Pop = pd.melt(Pop)
Pop.head()

0                  Alabama
1                   Alaska
2                  Arizona
3                 Arkansas
4               California
5                 Colorado
6              Connecticut
7                 Delaware
8     District of Columbia
9                  Florida
10                 Georgia
11                  Hawaii
12                   Idaho
13                Illinois
14                 Indiana
15                    Iowa
16                  Kansas
17                Kentucky
18               Louisiana
19                   Maine
20                Maryland
21           Massachusetts
22                Michigan
23               Minnesota
24             Mississippi
25                Missouri
26                 Montana
27                Nebraska
28                  Nevada
29           New Hampshire
30              New Jersey
31              New Mexico
32                New York
33          North Carolina
34            North Dakota
35                    Ohio
36                Oklahoma
3

,variable,value
0,Geographic Area,Alabama
1,Geographic Area,Alaska
2,Geographic Area,Arizona
3,Geographic Area,Arkansas
4,Geographic Area,California
